In [91]:
## Steps
# 1. Data cleaning and preprocessing
# 2. Train Test Split
# 3. BOW and TF-IDF ( to prevent Data leakage )
# 4. Model Training
# 5. Prediction

In [92]:
## Loading Data-Set
import pandas as pd
messages = pd.read_csv("SMSSpamCollection",sep="\t",names=["label","message"])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [93]:
## Data Cleaning and Preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

## creating instance of Porter Stemmer
stemmer = nltk.SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
corpus = []
def data_cleaning_fun(messages):
    for i in range(len(messages)):
        ## removing unnecessary alphabets
        review = re.sub("[^a-zA-z]"," ",messages["message"][i]) ## removing all special characters except a-z and A-Z by " "
        ## Lowering case
        review = review.lower()
        ## Splitting words
        review = review.split()
        ## Applying stemming and removing stop-words
        review = [stemmer.stem(word) for word in review if word not in stopwords.words("english")]
        ## Joining words to make sentence
        review = " ".join(review)
        ## Appending in list
        corpus.append(review)
    return corpus
## Calling function
data_cleaning_fun(messages)

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkts st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breath

In [95]:
## Corpus is the input feature
## Output Feature - Spam/Ham
y = pd.get_dummies(messages["label"],dtype=int)
y

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
5567,0,1
5568,1,0
5569,1,0
5570,1,0


In [96]:
## In output we can take only one feature from spam / ham column
## So we are taking spam column
y = y.iloc[:,1]
y

,spam
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


In [97]:
y.shape

(5572,)

In [98]:
## Bag of Words Model
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

## Applying train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size=20)

cv = CountVectorizer(max_features=2500,ngram_range=(1,2))
# Fit and transform on the original text data (corpus)
X_train = cv.fit_transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()
print("Shape of X = ",X_train.shape)
print("Vocubulary = ",cv.vocabulary_)

Shape of X =  (5552, 2500)
Vocubulary =  {'anyway': 76, 'mani': 1251, 'good': 832, 'even': 633, 'good even': 835, 'tell': 2072, 'friend': 743, 'plan': 1583, 'valentin': 2284, 'day': 498, 'lt': 1219, 'url': 2275, 'gt': 869, 'tell friend': 2074, 'valentin day': 2285, 'freemsg': 737, 'hi': 934, 'babi': 136, 'wow': 2445, 'got': 845, 'new': 1424, 'wanna': 2330, 'hot': 975, 'pic': 1572, 'fanci': 668, 'chat': 338, 'im': 1009, 'rpli': 1782, 'msg': 1382, 'rcv': 1699, 'got new': 849, 'aathi': 0, 'dear': 510, 'quick': 1679, 'done': 560, 'hand': 906, 'celebr': 323, 'full': 756, 'swing': 2048, 'yet': 2489, 'thk': 2111, 'shd': 1875, 'ok': 1495, 'said': 1797, 'plus': 1603, 'leav': 1142, 'line': 1170, 'sorri': 1945, 'help': 928, 'yep': 2482, 'pretti': 1651, 'doubl': 566, 'min': 1313, 'txts': 2215, 'orang': 1521, 'price': 1653, 'linerent': 1173, 'motorola': 1371, 'sonyericsson': 1943, 'free': 725, 'nokia': 1446, 'call': 252, 'mobileupd': 1352, 'optout': 1520, 'hv': 997, 'doubl min': 567, 'min txts': 13

In [99]:
## Model Training
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

spam_ham_detection_model_bow = GaussianNB()
## Training the model with training data
spam_ham_detection_model_bow.fit(X_train,y_train)
spam_ham_detection_model_bow

GaussianNB()

In [100]:
## Predicting Test data
y_pred = spam_ham_detection_model_bow.predict(X_test)
print(y_pred)

[0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0]


In [101]:
## Performance Matrix
from sklearn.metrics import accuracy_score,classification_report
print("Model Accuracy =" , accuracy_score(y_test,y_pred)*100)
print(classification_report(y_test,y_pred))

Model Accuracy = 85.0
              precision    recall  f1-score   support

           0       1.00      0.83      0.91        18
           1       0.40      1.00      0.57         2

    accuracy                           0.85        20
   macro avg       0.70      0.92      0.74        20
weighted avg       0.94      0.85      0.88        20



In [102]:
## USING TF-IDF Model
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

## Applying train test split
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size=20,random_state=42)

cv_tfidf = TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_train = cv_tfidf.fit_transform(X_train).toarray()
X_test = cv_tfidf.transform(X_test).toarray()
print("Shape of X = ",X_train.shape)
print("Vocubulary = ",cv_tfidf.vocabulary_)

Shape of X =  (5552, 2500)
Vocubulary =  {'yeah': 2475, 'leav': 1140, 'coupl': 454, 'minut': 1309, 'amp': 60, 'let': 1151, 'know': 1099, 'get': 780, 'mu': 1377, 'let know': 1152, 'custom': 484, 'place': 1573, 'call': 253, 'cud': 475, 'go': 804, 'pm': 1594, 'got': 853, 'way': 2349, 'contact': 427, 'guess': 891, 'worri': 2435, 'must': 1386, 'bodi': 195, 'quit': 1665, 'sure': 2031, 'take': 2045, 'slow': 1902, 'first': 706, 'test': 2078, 'guid': 892, 'relax': 1720, 'noth': 1449, 'said': 1781, 'reason': 1704, 'keep': 1084, 'nd': 1398, 'time': 2122, 'tri': 2174, 'pound': 1617, 'prize': 1642, 'claim': 360, 'easi': 597, 'per': 1542, 'bt': 228, 'nation': 1394, 'rate': 1680, 'nd time': 1400, 'time tri': 2126, 'tri contact': 2175, 'prize claim': 1644, 'claim easi': 363, 'easi call': 598, 'call per': 270, 'per minut': 1545, 'minut bt': 1310, 'bt nation': 230, 'nation rate': 1395, 'congratul': 422, 'still': 1978, 'look': 1197, 'job': 1065, 'much': 1378, 'ta': 2044, 'gud': 886, 'chikku': 347, 'care'

In [103]:
## Model Training with TF-IDF
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

spam_ham_detection_model_tfidf = GaussianNB()
## Training the model with training data
spam_ham_detection_model_tfidf.fit(X_train,y_train)
spam_ham_detection_model_tfidf

GaussianNB()

In [104]:
## Predicting Test data
y_pred = spam_ham_detection_model_tfidf.predict(X_test)
print(y_pred)

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0]


In [105]:
## Performance Matrix
from sklearn.metrics import accuracy_score,classification_report
print("Model Accuracy =" , accuracy_score(y_test,y_pred)*100)
print(classification_report(y_test,y_pred))

Model Accuracy = 90.0
              precision    recall  f1-score   support

           0       1.00      0.89      0.94        19
           1       0.33      1.00      0.50         1

    accuracy                           0.90        20
   macro avg       0.67      0.95      0.72        20
weighted avg       0.97      0.90      0.92        20

